# --- 0. CONFIGURATION ---

In [2]:
import os

# --- 1. CẤU HÌNH GỐC (ROOT) ---
# Dùng r'' để Python hiểu đây là đường dẫn Windows (không bị lỗi ký tự đặc biệt)
ROOT = r'E:\CAFA-6-Protein-Function-Prediction'

# --- 2. ĐỊNH NGHĨA CÁC THƯ MỤC CON (Tự động nối đuôi) ---
# os.path.join giúp nối đường dẫn đúng chuẩn cho cả Windows/Linux/Mac
INPUT_DIR  = os.path.join(ROOT, 'input')
OUTPUT_DIR = os.path.join(ROOT, 'output')
MODEL_DIR  = os.path.join(ROOT, 'models')

# Tạo sẵn thư mục output và models nếu chưa có (Tránh lỗi không lưu được file)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

print(f"✅ Đã cấu hình ROOT: {ROOT}")
print(f"📂 Input: {INPUT_DIR}")
print(f"📂 Output: {OUTPUT_DIR}")
print(f"📂 Models: {MODEL_DIR}")

✅ Đã cấu hình ROOT: E:\CAFA-6-Protein-Function-Prediction
📂 Input: E:\CAFA-6-Protein-Function-Prediction\input
📂 Output: E:\CAFA-6-Protein-Function-Prediction\output
📂 Models: E:\CAFA-6-Protein-Function-Prediction\models


In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
import os
from pathlib import Path

# TORCH MODULES
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchmetrics.classification import MultilabelF1Score

# --- 1. SETUP & CONFIGURATION ---

In [4]:
# --- CẤU HÌNH HỆ THỐNG & MODEL (UPDATED FOR 1D-CNN) ---

class config:
    # 1. Đường dẫn (Sử dụng ROOT đã định nghĩa ở cell đầu tiên)
    MAIN_DIR = os.path.join(ROOT, "input", "cafa-6-protein-function-prediction")
    
    # Đường dẫn cụ thể đến file Fasta (Tiện cho Dataset gọi)
    TRAIN_FASTA = os.path.join(MAIN_DIR, "Train", "train_sequences.fasta")
    TRAIN_LABELS = os.path.join(MAIN_DIR, "Train", "train_terms.tsv")
    TEST_FASTA = os.path.join(MAIN_DIR, "Test", "testsuperset.fasta")
    
    # 2. Tham số Model (Dành riêng cho xử lý chuỗi)
    num_labels = 500      # Số lượng GO Term cần dự đoán
    vocab_size = 25       # Số lượng ký tự Amino Acid (20 chuẩn + vài ký tự lạ/padding)
    embedding_dim = 128   # Kích thước vector nhúng nội bộ (Internal Embedding của CNN)
    max_len = 1024        # Độ dài chuỗi tối đa (Cắt hoặc Pad về độ dài này)
    
    # 3. Tham số Training
    n_epochs = 8
    
    # Batch size = 16 (Tối ưu cho card MX550 2GB/4GB VRAM)
    # Nếu chạy thử thấy mượt, có thể tăng lên 32. Nếu lỗi OOM thì giảm xuống 8.
    batch_size = 16       
    
    # Learning rate giảm nhẹ để model học ổn định với batch size nhỏ
    lr = 0.0005           
    
    # Tự động chọn GPU nếu có
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"✅ Đã tải cấu hình Config.")
print(f"🔹 Device: {config.device}")
print(f"🔹 Batch Size: {config.batch_size}")
print(f"🔹 Max Sequence Length: {config.max_len}")

✅ Đã tải cấu hình Config.
🔹 Device: cuda
🔹 Batch Size: 16
🔹 Max Sequence Length: 1024


# --- 2. DATA LOADING: PROTEIN SEQUENCE DATASET ---

In [5]:
from torch.utils.data import Dataset
import numpy as np
import pandas as pd
import torch
# Cần cài đặt thư viện biopython nếu chưa có: !pip install biopython
from Bio import SeqIO

# 1. Tạo bộ từ điển (Vocab) cho Axit Amin
AA_VOCAB = "ACDEFGHIKLMNPQRSTVWY"
aa_to_int = {aa: i+1 for i, aa in enumerate(AA_VOCAB)} # 1-20, 0 là padding

def encode_seq(seq, max_len=1024):
    seq = str(seq)
    encoded = [aa_to_int.get(aa, 0) for aa in seq]
    if len(encoded) > max_len:
        return encoded[:max_len]
    else:
        return encoded + [0] * (max_len - len(encoded))

class ProteinSequenceDataset(Dataset):
    def __init__(self, datatype, fasta_file, targets_file=None):
        self.datatype = datatype
        self.data = []
        
        # Đọc file FASTA (Sequence thô)
        print(f"Loading sequences from {fasta_file}...")
        for record in SeqIO.parse(fasta_file, "fasta"):
            self.data.append({
                "id": record.id,
                "seq": encode_seq(record.seq)
            })
            
        self.df = pd.DataFrame(self.data)
        
        # Nếu là Train thì load thêm nhãn (Targets)
        if datatype == "train" and targets_file:
            print("Loading targets...")
            labels = np.load(targets_file)
            # Giả sử thứ tự file npy khớp với fasta (cần cẩn thận đoạn này trong thực tế)
            # Để đơn giản cho starter, ta gán trực tiếp
            self.labels = labels[:len(self.df)] 

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        # Input là chuỗi số nguyên (VD: [1, 5, 2...])
        seq_tensor = torch.tensor(self.df.iloc[index]["seq"], dtype=torch.long)
        
        if self.datatype == "train":
            target = torch.tensor(self.labels[index], dtype=torch.float)
            return seq_tensor, target
        else:
            return seq_tensor, self.df.iloc[index]["id"]

# --- 3. MODEL ARCHITECTURE: 1D CONVOLUTIONAL NEURAL NETWORK (CNN) ---

In [6]:
class CNN1D(nn.Module):
    def __init__(self, num_classes, vocab_size=21, embed_dim=128):
        super(CNN1D, self).__init__()
        # Lớp này sẽ tự học đặc trưng từ con số (Axit amin)
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        
        # CNN quét trên các vector vừa học được
        self.conv_block1 = nn.Sequential(
            nn.Conv1d(in_channels=embed_dim, out_channels=32, kernel_size=9, padding=4),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(2)
        )
        self.conv_block2 = nn.Sequential(
            nn.Conv1d(32, 64, kernel_size=7, padding=3),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(2)
        )
        
        self.fc = nn.Sequential(
            nn.AdaptiveMaxPool1d(1), # Lấy đặc trưng mạnh nhất
            nn.Flatten(),
            nn.Linear(64, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        # x: [batch, seq_len] -> embed: [batch, seq_len, 128]
        x = self.embedding(x)
        # Đổi chiều cho Conv1D: [batch, 128, seq_len]
        x = x.permute(0, 2, 1) 
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        return self.fc(x)

# --- 4. MODEL TRAINING (SỬA ĐỔI CHO SEQUENCE CNN) ---

In [ ]:
# --- 4. MODEL TRAINING (SỬA ĐỔI CHO SEQUENCE CNN) ---

def train_model(train_size=0.9):
    
    # 1. Cấu hình đường dẫn file (Đảm bảo bạn đã Add dataset train_targets_top500)
    targets_path = F"{ROOT}/input/train_targets_top500/train_targets_top500.npy"
    
    # 2. Khởi tạo Dataset mới (Đọc từ Fasta)
    print("Initializing Dataset...")
    train_dataset = ProteinSequenceDataset(
        datatype="train", 
        fasta_file=Path(config.TRAIN_FASTA), 
        targets_file=targets_path
    )
    
    # 3. Chia tập Train/Val
    train_len = int(len(train_dataset) * train_size)
    val_len = len(train_dataset) - train_len
    train_set, val_set = random_split(train_dataset, [train_len, val_len])
    
    train_dataloader = DataLoader(train_set, batch_size=config.batch_size, shuffle=True)
    val_dataloader = DataLoader(val_set, batch_size=config.batch_size, shuffle=False)

    # 4. Khởi tạo Model CNN (Không cần input_dim từ ESM2 nữa)
    # vocab_size=21 (20 axit amin + 1 padding), embed_dim=128 (tùy chọn)
    model = CNN1D(num_classes=config.num_labels, vocab_size=21, embed_dim=128).to(config.device)

    optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)
    
    loss_fn = torch.nn.BCEWithLogitsLoss()
    
    # Metric F1
    f1_metric = MultilabelF1Score(num_labels=config.num_labels, average='macro').to(config.device)

    print(f"STARTING TRAINING on {config.device}...")
    
    best_val_f1 = 0.0
    best_threshold = 0.2 # Ngưỡng mặc định ban đầu

    for epoch in range(config.n_epochs):
        print(f"\nEPOCH {epoch+1}/{config.n_epochs}")
        
        # --- TRAIN PHASE ---
        model.train()
        total_train_loss = 0
        for seqs, targets in tqdm(train_dataloader, desc="Training"):
            seqs, targets = seqs.to(config.device), targets.to(config.device)
            
            optimizer.zero_grad()
            preds_logits = model(seqs)
            loss = loss_fn(preds_logits, targets)
            
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_dataloader)
        print(f"  Average Training Loss: {avg_train_loss:.4f}")

        # --- VALIDATION PHASE ---
        model.eval()
        all_val_preds = []
        all_val_targets = []
        with torch.no_grad():
            for seqs, targets in tqdm(val_dataloader, desc="Validation"):
                seqs, targets = seqs.to(config.device), targets.to(config.device)
                preds_logits = model(seqs)
                
                all_val_preds.append(torch.sigmoid(preds_logits))
                all_val_targets.append(targets)

        all_val_preds = torch.cat(all_val_preds)
        all_val_targets = torch.cat(all_val_targets)

        # Tìm ngưỡng (Threshold) tối ưu
        best_f1_epoch = 0
        best_thresh_epoch = 0
        thresholds = np.arange(0.1, 0.51, 0.05)
        
        # Chuyển metric về CPU để tránh lỗi bộ nhớ nếu GPU yếu
        f1_metric_cpu = f1_metric.to('cpu')
        all_val_preds_cpu = all_val_preds.to('cpu')
        all_val_targets_cpu = all_val_targets.to('cpu').int()

        for thresh in thresholds:
            f1_metric_cpu.threshold = thresh
            f1 = f1_metric_cpu(all_val_preds_cpu, all_val_targets_cpu)
            if f1 > best_f1_epoch:
                best_f1_epoch = f1
                best_thresh_epoch = thresh

        print(f"  Val F1: {best_f1_epoch:.4f} (Threshold: {best_thresh_epoch:.2f})")

        scheduler.step(best_f1_epoch)

        if best_f1_epoch > best_val_f1:
            best_val_f1 = best_f1_epoch
            best_threshold = best_thresh_epoch
            torch.save(model.state_dict(), f"{ROOT}/models/cnn_best.pth")
            print(f"  ⭐ New best model saved!")

    print("\nTRAINING FINISHED")
    print(f"Highest Validation F1-Score: {best_val_f1:.4f}")
    
    model.load_state_dict(
        torch.load(f"{ROOT}/models/cnn_best.pth", map_location=config.device, weights_only=True)
    )
    return model, best_threshold

# Gọi hàm train (Không cần truyền tham số ESM2 nữa)
cnn_model, best_threshold = train_model()

Initializing Dataset...
Loading sequences from E:\CAFA-6-Protein-Function-Prediction\input\cafa-6-protein-function-prediction\Train\train_sequences.fasta...
Loading targets...
STARTING TRAINING on cuda...

EPOCH 1/8


Training: 100%|██████████| 4636/4636 [00:49<00:00, 93.35it/s] 


  Average Training Loss: 0.1875


Validation: 100%|██████████| 516/516 [00:02<00:00, 242.94it/s]


  Val F1: 0.0459 (Threshold: 0.10)
  ⭐ New best model saved!

EPOCH 2/8


...

Training: 100%|██████████| 4636/4636 [06:58<00:00, 11.09it/s]


  Average Training Loss: 0.1841


Validation: 100%|██████████| 516/516 [00:12<00:00, 40.31it/s]


  Val F1: 0.0532 (Threshold: 0.10)

TRAINING FINISHED
Highest Validation F1-Score: 0.0534


C:\Users\admin\AppData\Local\Temp\ipykernel_18640\2864222538.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f"{ROOT}/models/cnn_best.pth", map_location=con

# --- 5. GENERATING PREDICTIONS ---

In [8]:
# --- 5. GENERATING PREDICTIONS (UPDATED FOR SEQUENCE CNN) ---

def predict(model, threshold):
    # 1. Cấu hình đường dẫn file Test (File Sequence thô)
    # Lưu ý: Kiểm tra lại đường dẫn này nếu bạn lưu ở chỗ khác
    test_fasta_path = f"{config.MAIN_DIR}/Test/testsuperset.fasta"
    
    print(f"Loading Test Data from: {test_fasta_path}")
    
    # 2. Khởi tạo Dataset (Sử dụng class mới đã sửa ở trên)
    test_dataset = ProteinSequenceDataset(datatype="test", fasta_file=test_fasta_path)
    test_dataloader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False)
    
    model.eval()
    
    # 3. Lấy danh sách tên nhãn (GO Terms)
    # Cần load lại file train_terms để biết cột 0 là nhãn gì, cột 1 là nhãn gì...
    labels_path = f"{config.MAIN_DIR}/Train/train_terms.tsv"
    labels_df = pd.read_csv(labels_path, sep="\t")
    top_terms = labels_df.groupby("term")["EntryID"].count().sort_values(ascending=False)
    labels_names = top_terms.head(config.num_labels).index.values
    
    print("\nGENERATING PREDICTIONS FOR THE TEST SET...")
    
    results = []
    with torch.no_grad():
        for seqs, ids in tqdm(test_dataloader, desc="Predicting"):
            seqs = seqs.to(config.device)
            
            # Đưa qua model
            preds_logits = model(seqs)
            preds_probs = torch.sigmoid(preds_logits).cpu().numpy()
            
            # Lọc kết quả theo ngưỡng (Threshold)
            for i, protein_id in enumerate(ids):
                protein_probs = preds_probs[i]
                # Chỉ lấy những nhãn có xác suất > threshold
                go_indices = np.where(protein_probs > threshold)[0]
                
                for idx in go_indices:
                    results.append({
                        "Id": protein_id,
                        "GO term": labels_names[idx],
                        "Confidence": protein_probs[idx]
                    })
    
    submission_df = pd.DataFrame(results)
    print("PREDICTIONS COMPLETE.")
    return submission_df

# Gọi hàm predict với model CNN vừa train xong
# Lưu ý: Thay 'ems2_model' cũ bằng 'cnn_model'
submission_df = predict(cnn_model, best_threshold)

Loading Test Data from: E:\CAFA-6-Protein-Function-Prediction\input\cafa-6-protein-function-prediction/Test/testsuperset.fasta
Loading sequences from E:\CAFA-6-Protein-Function-Prediction\input\cafa-6-protein-function-prediction/Test/testsuperset.fasta...

GENERATING PREDICTIONS FOR THE TEST SET...


Predicting: 100%|██████████| 14020/14020 [07:43<00:00, 30.27it/s]


PREDICTIONS COMPLETE.


# --- 6. SUBMISSION FILE GENERATION ---

In [9]:
import os
import pandas as pd

# --- 1. CẤU HÌNH ĐƯỜNG DẪN (Dựa trên Tree của bạn) ---
# Vì notebook đang ở trong '1_train_models', ta dùng '../' để ra thư mục gốc
blast_path = f"{ROOT}/input/blast-quick-sprof-zero-pred/submission.tsv"
output_path = f"{ROOT}/output/submission_2.tsv"

print("\n--- BẮT ĐẦU TẠO FILE SUBMISSION (CNN + BLAST FILL) ---")

# --- 2. LOAD DATA ---
if os.path.exists(blast_path):
    print(f"✅ Đã tìm thấy file BLAST tại: {blast_path}")
    
    # Load file BLAST (Header=None vì file này thường không có dòng tiêu đề)
    submission_blast = pd.read_csv(blast_path, sep='\t', header=None, names=['Id', 'GO term', 'Confidence_Blast'])
    
    # Load kết quả từ CNN (submission_df là biến có sẵn từ cell trên)
    # Đổi tên cột Confidence thành Confidence_CNN để dễ phân biệt
    submission_cnn = submission_df.rename(columns={'Confidence': 'Confidence_CNN'})

    # --- 3. MERGE VÀ XỬ LÝ LOGIC ---
    print("⏳ Đang trộn kết quả (Merge)...")
    # Outer join để giữ lại tất cả các cặp (Protein, GO Term) từ cả 2 nguồn
    subs = pd.merge(submission_cnn, submission_blast, on=['Id', 'GO term'], how='outer')

    # LOGIC QUAN TRỌNG: "Ưu tiên CNN, thiếu mới dùng BLAST"
    # Mục tiêu: File này phải mang đặc trưng của CNN để phục vụ Ensemble sau này.
    # fillna: Nếu CNN có điểm -> lấy CNN. Nếu CNN là NaN (không dự đoán) -> lấy BLAST.
    subs['Confidence_Final'] = subs['Confidence_CNN'].fillna(subs['Confidence_Blast'])
    
    # Chọn cột cuối cùng
    final_df = subs[['Id', 'GO term', 'Confidence_Final']]

else:
    print("⚠️ CẢNH BÁO: Không tìm thấy file BLAST! Chỉ lưu kết quả thuần của CNN.")
    final_df = submission_df.rename(columns={'Confidence': 'Confidence_Final'})

# --- 4. LƯU FILE ---
# Tạo thư mục output nếu chưa có
os.makedirs(f"{ROOT}/output", exist_ok=True)

# Lưu file (không index, không header để đúng format cuộc thi)
final_df.to_csv(output_path, sep='\t', header=False, index=False)

print("="*40)
print(f"🎉 THÀNH CÔNG! File đã được lưu tại: {output_path}")
print(f"📊 Tổng số dự đoán: {len(final_df)}")
print("="*40)


--- BẮT ĐẦU TẠO FILE SUBMISSION (CNN + BLAST FILL) ---
✅ Đã tìm thấy file BLAST tại: E:\CAFA-6-Protein-Function-Prediction/input/blast-quick-sprof-zero-pred/submission.tsv
⏳ Đang trộn kết quả (Merge)...
🎉 THÀNH CÔNG! File đã được lưu tại: E:\CAFA-6-Protein-Function-Prediction/output/submission_2.tsv
📊 Tổng số dự đoán: 28968486
